In [1]:
mountedDB = {}

## Midterm Project: Quant Modeling for Data Scientists

In [2]:
# 安装依赖
print("开始安装依赖包...")

!pip install seaborn -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install scikit-learn -i https://pypi.tuna.tsinghua.edu.cn/simple/  
!pip install statsmodels -i https://pypi.tuna.tsinghua.edu.cn/simple/

print("依赖包安装完成！")

开始安装依赖包...
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     |████████████████████████████████| 294 kB 12.3 MB/s            
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     |████████████████████████████████| 13.5 MB 48.1 MB/s            
     |████████████████████████████████| 308 kB 116.6 MB/s            
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     |████████████████████████████████| 10.7 MB 22.5 MB/s            
     |████████████████████████████████| 233 kB 103.4 MB/s            
依赖包安装完成！


In [3]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

****  

### 1. Data Processing  
  - data analysis, data clean, and data preprocessing (Please refer to  Ch2 codes from our textbook and my slides)  
  - delete the features with data leakage issues such as community price in the housing project  
  - divide the sample into 80% training and 20% testing by using sklearn.model_selection.train_test_split (random_state==111)

In [4]:
df_tr_price = pd.read_csv('/home/mw/input/hackathon255769/ruc_Class25Q2_train_price.csv', low_memory=False)
df_tr_rent = pd.read_csv('/home/mw/input/hackathon255769/ruc_Class25Q2_train_rent.csv', low_memory=False)
df_te_price = pd.read_csv('/home/mw/input/hackathon255769/ruc_Class25Q2_test_price.csv', low_memory=False)
df_te_rent = pd.read_csv('/home/mw/input/hackathon255769/ruc_Class25Q2_test_rent.csv', low_memory=False)

#### 1.1 概览数据并检查缺失值

In [5]:
df_tr_rent.sample(3)

,城市,户型,装修,Price,楼层,面积,朝向,交易时间,付款方式,租赁方式,...,供水,供暖,供电,燃气费,供热费,停车位,停车费用,coord_x,coord_y,客户反馈
46022,3,2室1厅1卫,NaN,227409.710416,低楼层/18层,97.00㎡,南,2024-11-04,季付价,整租,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.076579,32.258860,相比较之下，租期灵活，反正，小区道路宽度标准，简单总结，健身设施常规
11580,0,2室1厅1卫,精装修,828340.373976,6/6层,69.53㎡,南,2024-10-30,季付价,整租,...,民水,集中供暖,民电,2.61元/m³,30元/㎡,500.0,150,117.556512,40.935260,布局合理，说来也是，打车方便，房龄一般
48283,3,1室1厅1卫,NaN,251422.331586,中楼层/9层,40.00㎡,南 北,2025-02-05,季付价,整租,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.666862,32.419286,骑行方便，就周边而言，网络稳定


In [6]:
def explore_dataframe(df, dataset_name):
    """探索数据框的基本信息"""
    print(f"====================== {dataset_name} 数据探索 ======================")

    print("\n基本信息:")
    print(df.info())
    
    print("\n缺失值统计:")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        '缺失数量': missing_data,
        '缺失比例%': missing_percent
    })
    print(missing_df.sort_values('缺失比例%', ascending=False))
    high_missing_columns = missing_df[missing_df['缺失比例%'] > 50].index.tolist()
    
    return high_missing_columns


In [7]:
high_missing_price_columns = explore_dataframe(df_tr_price, 'train_price')
high_missing_rent_columns = explore_dataframe(df_tr_rent, 'train_rent')

====================== train_price 数据探索 ======================

基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103871 entries, 0 to 103870
Data columns (total 55 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   城市         103871 non-null  int64  
 1   区域         103871 non-null  float64
 2   板块         103871 non-null  float64
 3   环线         40419 non-null   object 
 4   Price      103871 non-null  float64
 5   房屋户型       103291 non-null  object 
 6   所在楼层       103871 non-null  object 
 7   建筑面积       103871 non-null  object 
 8   套内面积       35984 non-null   object 
 9   房屋朝向       103870 non-null  object 
 10  建筑结构       103291 non-null  object 
 11  装修情况       103291 non-null  object 
 12  梯户比例       101252 non-null  object 
 13  配备电梯       91520 non-null   object 
 14  别墅类型       1443 non-null    object 
 15  交易时间       103871 non-null  object 
 16  交易权属       103871 non-null  object 
 17  上次交易       78422 non-null   object 

In [8]:
print(high_missing_price_columns,high_missing_rent_columns)

['环线', '套内面积', '别墅类型', '抵押信息', '户型介绍', '环线位置', '物业办公电话', '供暖', '供热费'] ['装修', '车位', '采暖', '环线位置', '物业办公电话', '供暖', '供热费']


#### 1.2 数据预处理  
  - 初步筛选列（缺失比例过大/数据泄露/重复信息）  
  - 提取并初步处理数字信息  
  - 填充缺失值

In [9]:
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

def extract_numeric_features(df):
    """提取数字特征"""
    df_processed = df.copy()
    
    numeric_keywords = ['率', '费', '面积', '物业费', '停车费', '总数', '建筑年代']
    object_cols = df_processed.select_dtypes(include=['object']).columns
    cols_to_process = [col for col in object_cols if any(keyword in col for keyword in numeric_keywords)]
    
    for col in cols_to_process:
        def extract_numeric_value(value):
            if pd.isna(value):
                return np.nan
            str_value = str(value).strip()
            if any(na_word in str_value for na_word in ['暂无', '未知', '无', '没有']):
                return np.nan
            if '-' in str_value and any(char.isdigit() for char in str_value):
                parts = str_value.split('-')
                numbers = []
                for part in parts:
                    found = re.findall(r"[-+]?\d*\.\d+|\d+", part)
                    if found:
                        numbers.append(float(found[0]))
                if len(numbers) == 2:
                    return sum(numbers) / 2
                elif numbers:
                    return numbers[0]
            numbers = re.findall(r"[-+]?\d*\.\d+|\d+", str_value)
            return float(numbers[0]) if numbers else np.nan
        
        df_processed[col] = df_processed[col].apply(extract_numeric_value)
    
    return df_processed

def geo_cluster_imputation(df):
    """
    基于经纬度进行地理聚类填充
    """
    df_imputed = df.copy()
    n_clusters = 100  # 固定100个聚类
    
    # 检查经纬度列
    if 'lon' not in df_imputed.columns or 'lat' not in df_imputed.columns:
        raise ValueError("数据必须包含 'lon' 和 'lat' 列")
    
    # 处理经纬度缺失值（用均值填充）
    if df_imputed['lon'].isnull().sum() > 0:
        df_imputed['lon'] = df_imputed['lon'].fillna(df_imputed['lon'].mean())
    if df_imputed['lat'].isnull().sum() > 0:
        df_imputed['lat'] = df_imputed['lat'].fillna(df_imputed['lat'].mean())
    
    # 准备经纬度数据
    X_geo = df_imputed[['lon', 'lat']].to_numpy()
    
    # 地理聚类（固定100个聚类）
    print("正在进行地理聚类...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    clusters = kmeans.fit_predict(X_geo)
    
    print("开始填充缺失值...")
    
    # 特殊文本列，直接填充'unknown'
    special_text_cols = ['房屋年限','房屋优势','周边配套', '交通出行', '配套设施','上次交易']
    
    # 为每个聚类预先计算填充值
    cluster_fill_values = {}
    
    for cluster_id in range(n_clusters):
        cluster_indices = df_imputed.index[clusters == cluster_id]
        cluster_data = df_imputed.loc[cluster_indices]
        
        fill_values = {}
        for col in df_imputed.columns:
            if df_imputed[col].isnull().sum() > 0:
                # 特殊文本列直接填充'unknown'
                if col in special_text_cols:
                    fill_values[col] = 'unknown'
                else:
                    cluster_non_missing = cluster_data[col].dropna()
                    
                    if len(cluster_non_missing) > 0:
                        # 聚类内有数据：用聚类统计量
                        if df_imputed[col].dtype in [np.number]:
                            fill_values[col] = cluster_non_missing.mean()
                        else:
                            # 类别列：用聚类内众数的第一个值
                            modes = cluster_non_missing.mode()
                            fill_values[col] = modes.iloc[0] if len(modes) > 0 else 'unknown'
                    else:
                        # 聚类内无数据：用全局统计量
                        global_non_missing = df_imputed[col].dropna()
                        if len(global_non_missing) > 0:
                            if df_imputed[col].dtype in [np.number]:
                                fill_values[col] = global_non_missing.median()
                            else:
                                global_modes = global_non_missing.mode()
                                fill_values[col] = global_modes.iloc[0] if len(global_modes) > 0 else 'unknown'
                        else:
                            fill_values[col] = 'unknown'
        
        cluster_fill_values[cluster_id] = fill_values
    
    # 批量填充缺失值（按列批量操作）
    for col in df_imputed.columns:
        if df_imputed[col].isnull().sum() > 0:
            filled_count = 0
            
            # 特殊文本列直接批量填充
            if col in special_text_cols:
                missing_count = df_imputed[col].isnull().sum()
                df_imputed[col] = df_imputed[col].fillna('unknown')
                filled_count = missing_count
            else:
                # 创建该列的填充掩码
                missing_mask = df_imputed[col].isnull()
                
                # 对每个聚类批量填充
                for cluster_id in range(n_clusters):
                    cluster_indices = df_imputed.index[clusters == cluster_id]
                    
                    # 找出该聚类中需要填充的行
                    cluster_missing_indices = cluster_indices.intersection(df_imputed.index[missing_mask])
                    
                    if len(cluster_missing_indices) > 0:
                        fill_value = cluster_fill_values[cluster_id].get(col)
                        if fill_value is not None:
                            df_imputed.loc[cluster_missing_indices, col] = fill_value
                            filled_count += len(cluster_missing_indices)
            
            print(f"{col}列已完成填充，共填充{filled_count}个缺失值")
    
    return df_imputed

def preprocess_data(df):
    """主预处理函数"""
    print("开始数据预处理...")
    
    # 提取数字特征
    print("提取数字特征...")
    df_processed = extract_numeric_features(df)
    
    # 基于经纬度聚类填充
    df_filled = geo_cluster_imputation(df_processed)
    
    print(f"\n预处理完成!")
    print(f"总样本数: {len(df_filled)}")
    print(f"地理区域数: 100")
    
    return df_filled

##### a.处理售价数据框得到df_price_preprocessed

In [10]:
data_leakage_columns = ['户型介绍','核心卖点','客户反馈'] # 可能存在数据泄露的特征
repeat_columns = ['板块_comm','区县','coord_x','coord_y','建筑结构_comm'] # 重复的特征
price_cols_to_drop =list(set(data_leakage_columns + high_missing_price_columns + repeat_columns))

df_tr_price_useful = df_tr_price.drop(columns=price_cols_to_drop) 

df_price_preprocessed = preprocess_data(df_tr_price_useful)

开始数据预处理...
提取数字特征...
正在进行地理聚类...
开始填充缺失值...
房屋户型列已完成填充，共填充580个缺失值
房屋朝向列已完成填充，共填充1个缺失值
建筑结构列已完成填充，共填充580个缺失值
装修情况列已完成填充，共填充580个缺失值
梯户比例列已完成填充，共填充2619个缺失值
配备电梯列已完成填充，共填充12351个缺失值
上次交易列已完成填充，共填充25449个缺失值
房屋用途列已完成填充，共填充1个缺失值
房屋年限列已完成填充，共填充44510个缺失值
房屋优势列已完成填充，共填充21236个缺失值
周边配套列已完成填充，共填充43138个缺失值
交通出行列已完成填充，共填充43264个缺失值
物业类别列已完成填充，共填充28413个缺失值
建筑年代列已完成填充，共填充35101个缺失值
开发商列已完成填充，共填充36141个缺失值
房屋总数列已完成填充，共填充7131个缺失值
楼栋总数列已完成填充，共填充7131个缺失值
物业公司列已完成填充，共填充37908个缺失值
绿 化 率列已完成填充，共填充32883个缺失值
容 积 率列已完成填充，共填充33154个缺失值
物 业 费列已完成填充，共填充31158个缺失值
产权描述列已完成填充，共填充28413个缺失值
供水列已完成填充，共填充30298个缺失值
供电列已完成填充，共填充30292个缺失值
燃气费列已完成填充，共填充32701个缺失值
停车位列已完成填充，共填充34568个缺失值
停车费用列已完成填充，共填充38648个缺失值

预处理完成!
总样本数: 103871
地理区域数: 100


In [11]:
df_price_preprocessed.describe()

,城市,区域,板块,Price,建筑面积,lon,lat,年份,建筑年代,房屋总数,楼栋总数,绿 化 率,容 积 率,物 业 费,燃气费,停车位,停车费用
count,103871.000000,103871.000000,103871.000000,1.038710e+05,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000,103871.000000
mean,3.816272,65.564989,604.388405,2.262366e+06,99.236130,114.790564,32.181145,2020.913200,2007.361266,1898.682747,29.952790,37.999600,2.698042,2.453974,2.683961,1054.232693,313.389233
std,3.356300,33.694763,334.938598,2.532925e+06,46.793589,5.837851,5.955405,0.845009,7.488489,1815.769250,58.983175,225.107902,1.361084,3.351938,0.571056,1107.972118,202.668007
min,0.000000,3.000000,1.000000,7.455330e+04,11.700000,103.507243,23.025853,2015.000000,1933.000000,1.000000,1.000000,0.010000,0.020000,0.020000,0.400000,1.000000,0.000000
25%,2.000000,35.000000,314.000000,8.910913e+05,70.980000,107.620335,30.379362,2021.000000,2005.000000,734.000000,7.000000,30.000000,2.000000,1.471377,2.150000,480.000000,155.412844
50%,3.000000,68.000000,602.000000,1.479407e+06,91.110000,115.552921,32.136038,2021.000000,2008.500000,1372.000000,15.000000,34.000000,2.500000,1.900000,2.610000,734.000000,300.000000
75%,6.000000,88.000000,909.000000,2.680757e+06,118.760000,118.046991,37.699695,2021.000000,2012.000000,2429.000000,29.291824,35.000000,3.000000,2.630000,3.000000,1300.000000,387.984141
max,11.000000,131.000000,1186.000000,5.622643e+07,508.110000,122.966669,42.189559,2022.000000,2021.500000,12669.000000,734.000000,10500.000000,30.000000,76.450000,5.000000,8700.000000,2300.000000


In [12]:
df_price_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103871 entries, 0 to 103870
Data columns (total 39 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   城市      103871 non-null  int64  
 1   区域      103871 non-null  float64
 2   板块      103871 non-null  float64
 3   Price   103871 non-null  float64
 4   房屋户型    103871 non-null  object 
 5   所在楼层    103871 non-null  object 
 6   建筑面积    103871 non-null  float64
 7   房屋朝向    103871 non-null  object 
 8   建筑结构    103871 non-null  object 
 9   装修情况    103871 non-null  object 
 10  梯户比例    103871 non-null  object 
 11  配备电梯    103871 non-null  object 
 12  交易时间    103871 non-null  object 
 13  交易权属    103871 non-null  object 
 14  上次交易    103871 non-null  object 
 15  房屋用途    103871 non-null  object 
 16  房屋年限    103871 non-null  object 
 17  产权所属    103871 non-null  object 
 18  房屋优势    103871 non-null  object 
 19  周边配套    103871 non-null  object 
 20  交通出行    103871 non-null  object 
 21  lon     10

##### b.处理租金数据框得到df_rent_preprocessed

In [13]:
data_leakage_columns = ['客户反馈'] # 可能存在数据泄露的特征
repeat_columns = ['coord_x','coord_y','供电','供水'] # 重复的特征
rent_cols_to_drop =list(set(data_leakage_columns + high_missing_rent_columns + repeat_columns))

df_tr_rent_useful = df_tr_rent.drop(columns=rent_cols_to_drop) 
df_tr_rent_useful['装修'] = df_tr_rent['装修'].fillna('非精装修') # 补一下装修信息 
payway = ["半年付价", "季付价", "年付价", "双月付价", "月付价"]
df_tr_rent_useful.loc[~df_tr_rent_useful['付款方式'].isin(payway), '付款方式'] = '季付价' # 删点怪东西

df_rent_preprocessed = preprocess_data(df_tr_rent_useful)

开始数据预处理...
提取数字特征...
正在进行地理聚类...
开始填充缺失值...
户型列已完成填充，共填充1个缺失值
楼层列已完成填充，共填充5个缺失值
朝向列已完成填充，共填充5个缺失值
电梯列已完成填充，共填充4个缺失值
用水列已完成填充，共填充17740个缺失值
用电列已完成填充，共填充17324个缺失值
燃气列已完成填充，共填充4582个缺失值
租期列已完成填充，共填充46933个缺失值
配套设施列已完成填充，共填充30451个缺失值
区县列已完成填充，共填充4677个缺失值
板块列已完成填充，共填充5144个缺失值
物业类别列已完成填充，共填充20124个缺失值
建筑年代列已完成填充，共填充26149个缺失值
开发商列已完成填充，共填充26984个缺失值
房屋总数列已完成填充，共填充4631个缺失值
楼栋总数列已完成填充，共填充4631个缺失值
物业公司列已完成填充，共填充28753个缺失值
绿 化 率列已完成填充，共填充24402个缺失值
容 积 率列已完成填充，共填充24080个缺失值
物 业 费列已完成填充，共填充22159个缺失值
建筑结构列已完成填充，共填充20741个缺失值
产权描述列已完成填充，共填充20124个缺失值
燃气费列已完成填充，共填充25057个缺失值
停车位列已完成填充，共填充25479个缺失值
停车费用列已完成填充，共填充28476个缺失值

预处理完成!
总样本数: 98899
地理区域数: 100


In [14]:
df_rent_preprocessed.describe()

,城市,Price,面积,lon,lat,年份,区县,板块,建筑年代,房屋总数,楼栋总数,绿 化 率,容 积 率,物 业 费,燃气费,停车位,停车费用
count,98899.000000,9.889900e+04,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000,98899.000000
mean,4.322157,5.829090e+05,80.506477,115.752394,31.420651,2021.643010,70.348988,589.641366,2007.002343,2050.836409,28.039986,43.414102,2.942460,2.713851,2.909711,1225.219142,317.200353
std,3.321254,6.218424e+05,42.643602,5.578828,6.378794,0.479114,35.343485,337.200341,8.604208,1886.000270,56.997791,320.977699,1.570196,3.453506,0.548078,1414.826045,226.351989
min,0.000000,1.793807e+04,6.050000,103.482606,23.025849,2021.000000,3.000000,1.000000,1936.000000,1.000000,1.000000,0.010000,0.020000,0.020000,0.400000,1.000000,0.000000
25%,2.000000,2.416282e+05,50.240000,114.357093,24.201337,2021.000000,38.000000,303.000000,2004.000000,778.000000,6.000000,30.000000,2.000000,1.450000,2.610000,418.618986,150.000000
50%,3.000000,3.949369e+05,79.000000,117.156819,32.163991,2022.000000,70.000000,585.000000,2008.000000,1446.000000,13.000000,35.000000,2.670000,2.200000,2.950000,779.000000,300.000000
75%,7.000000,7.248954e+05,99.800000,121.644325,37.606197,2022.000000,94.000000,885.000000,2013.000000,2642.000000,27.000000,36.000000,3.300000,3.000000,3.450000,1450.000000,380.000000
max,11.000000,1.540419e+07,440.000000,122.966660,42.189642,2022.000000,131.000000,1186.000000,2022.000000,12669.000000,734.000000,10500.000000,30.000000,76.450000,5.000000,8700.000000,2300.000000


In [15]:
df_rent_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98899 entries, 0 to 98898
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      98899 non-null  int64  
 1   户型      98899 non-null  object 
 2   Price   98899 non-null  float64
 3   楼层      98899 non-null  object 
 4   面积      98899 non-null  float64
 5   朝向      98899 non-null  object 
 6   交易时间    98899 non-null  object 
 7   付款方式    98899 non-null  object 
 8   租赁方式    98899 non-null  object 
 9   电梯      98899 non-null  object 
 10  用水      98899 non-null  object 
 11  用电      98899 non-null  object 
 12  燃气      98899 non-null  object 
 13  租期      98899 non-null  object 
 14  配套设施    98899 non-null  object 
 15  lon     98899 non-null  float64
 16  lat     98899 non-null  float64
 17  年份      98899 non-null  float64
 18  区县      98899 non-null  float64
 19  板块      98899 non-null  float64
 20  物业类别    98899 non-null  object 
 21  建筑年代    98899 non-null  float64
 22

#### 1.3 划分训练集和测试集

In [16]:
def prepare_features_target(df, droplist, target_col='Price'):
    """准备特征和目标变量"""
    # 移除不必要的列 
    features = df.drop(columns=[target_col] + droplist)
    target = df[target_col]
    
    return features, target

def split_data(features, target, test_size=0.2, random_state=111):
    """划分训练集和测试集"""
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=test_size, random_state=random_state
    )
    
    print(f"训练集形状: {X_train.shape}")
    print(f"测试集形状: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test

In [17]:
df_price_preprocessed['所在楼层'].value_counts()

所在楼层
高楼层 (共6层)     5132
中楼层 (共6层)     4211
低楼层 (共18层)    2867
低楼层 (共6层)     2752
高楼层 (共18层)    2676
              ... 
底层 (共31层)        1
中楼层 (共63层)       1
低楼层 (共60层)       1
地下室 (共17层)       1
地下室 (共29层)       1
Name: count, Length: 258, dtype: int64

In [18]:
# 准备房价数据
# unwanted_fe 感觉没用/暂时不想处理/肉眼共线
unwanted_price_fe = ['物业公司','开发商','物业类别','上次交易','梯户比例','板块','区域','交易时间','交易权属','产权描述']
X_price, y_price = prepare_features_target(df_price_preprocessed, unwanted_price_fe)
X_train_price, X_test_price, y_train_price, y_test_price = split_data(X_price, y_price)

训练集形状: (83096, 28)
测试集形状: (20775, 28)


In [19]:
# 准备租金数据
# unwanted_fe 感觉没用/暂时不想处理/肉眼共线
unwanted_rent_fe = ['租期','物业公司','开发商','物业类别','产权描述','板块','区县','楼层','交易时间']
X_rent, y_rent = prepare_features_target(df_rent_preprocessed, unwanted_rent_fe)
X_train_rent, X_test_rent, y_train_rent, y_test_rent = split_data(X_rent, y_rent)

训练集形状: (79119, 25)
测试集形状: (19780, 25)


In [20]:
print(df_price_preprocessed.shape)
print(len(unwanted_price_fe))

(103871, 39)
10


In [21]:
print(df_rent_preprocessed.shape)
print(len(unwanted_rent_fe))

(98899, 35)
9


### 2. Feature Engineering  
  - create new features: log transformation, polynomial features, interaction terms, binning, dummies, and etc.  
  - feature selection: use correlation, VIF, Lasso and etc. to select the features  
  - outlier detection and treatment: use IQR, Z-score, and etc. 

#### 2.1 处理并创建新特征

In [22]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures


def process_floor_info(df):
    """处理楼层信息"""
    if '所在楼层' in df.columns:
        # 删除括号及括号内的内容
        df['所在楼层'] = df['所在楼层'].str.replace(r'\(.*\)', '', regex=True)
        df['所在楼层'] = df['所在楼层'].str.strip()
 
    if '楼层' in df.columns:
        # 处理有斜杠的情况：提取斜杠后的数字
        df['楼层'] = df['楼层'].str.replace(r'.*/(\d+)层', r'\1', regex=True)
        # 处理无斜杠的情况：直接提取数字
        df['楼层'] = df['楼层'].str.replace(r'(\d+)层', r'\1', regex=True)
    
    return df

def extract_room_layout(df):
    """提取户型信息到室、厅、卫、厨列"""
    
    # 确定列名
    layout_col = '房屋户型' if '房屋户型' in df.columns else '户型'
    
    # 提取数字信息为整数
    df['室'] = df[layout_col].str.extract(r'(\d+)(?:室|房间|居室)').fillna(0).astype(float)
    df['厅'] = df[layout_col].str.extract(r'(\d+)厅').fillna(0).astype(float)
    df['卫'] = df[layout_col].str.extract(r'(\d+)卫').fillna(0).astype(float)
    df['厨'] = df[layout_col].str.extract(r'(\d+)厨').fillna(0).astype(float)
    
    # 删除原列
    df = df.drop(columns=layout_col)
    
    return df

def classify_property_usage(df):
    """房屋用途分类"""
    if '房屋用途' in df.columns:
        def categorize_usage(usage):
            if pd.isna(usage):
                return '其他'
            usage_str = str(usage)
            
            # 豪宅
            mansion_keywords = ['别墅', '四合院', '花园洋房']
            if any(keyword in usage_str for keyword in mansion_keywords):
                return '豪宅'

            # 住宅
            residential_keywords = ['普通住宅','新式里弄']
            if any(keyword in usage_str for keyword in residential_keywords):
                return '住宅'
            
            # 公寓
            if '公寓' in usage_str:
                return '公寓'
            
            # 商住混合
            if '商住两用' in usage_str:
                return '商住混合'
            
            # 商业
            commercial_keywords = ['写字楼', '商']
            if any(keyword in usage_str for keyword in commercial_keywords):
                return '商业'
            
            return '其他'
        
        df['房屋用途'] = df['房屋用途'].apply(categorize_usage)
    
    return df

def extract_orientation_features(df):
    """提取朝向信息 - 简化版本"""
    
    orientation_col = None
    for col in ['朝向', '房屋朝向']:
        if col in df.columns:
            orientation_col = col
            break
    
    if orientation_col is None:
        return df

    # 使用str.get_dummies自动创建朝向列
    dummies = df[orientation_col].str.get_dummies(sep=' ')
    
    # 删除"未知"列
    if '未知' in dummies.columns:
        dummies = dummies.drop('未知', axis=1)
    
    # 直接定义方向列表
    directions = ['东', '南', '西', '北', '东南', '东北', '西南', '西北']
    
    # 为每个方向创建列
    for direction in directions:
        if direction in dummies.columns:
            df[direction] = dummies[direction].astype(bool)
        else:
            df[direction] = False
    
    # 计算朝向个数（现在不会计入"未知"）
    df['朝向个数'] = dummies[directions].sum(axis=1)
    
    # 判断南北通透
    df['南北通透'] = (dummies.get('南', 0) & dummies.get('北', 0)).astype(bool)

    return df

def extract_transport_features(df):
    """提取交通出行和房屋优势中的交通信息为布尔值"""
    
    # 创建新列，初始值为False
    df['地铁'] = False
    df['公交'] = False
    
    # 处理房屋优势列 - 地铁信息
    if '房屋优势' in df.columns:
        # 房屋优势中含"地铁"则地铁列设为True
        df.loc[df['房屋优势'].str.contains('地铁', na=False), '地铁'] = True
    
    # 处理交通出行列
    if '交通出行' in df.columns:
        # 交通出行中含"公交"则公交列设为True
        df.loc[df['交通出行'].str.contains('公交', na=False), '公交'] = True
        
        # 交通出行中含"轻轨"或"轨道"或"地铁"则地铁列设为True
        metro_mask = df['交通出行'].str.contains('轻轨|轨道|地铁', na=False)
        df.loc[metro_mask, '地铁'] = True
    
    return df

def complete_property_age(df):
    """用房屋优势信息补全房屋年限的unknown值"""
    
    if '房屋年限' in df.columns and '房屋优势' in df.columns:
        # 找到房屋年限为'unknown'的行
        unknown_mask = df['房屋年限'] == 'unknown'
        
        # 用房屋优势信息补全
        if unknown_mask.any():
            # 检查是否包含"房本满五年"
            five_year_mask = df['房屋优势'].str.contains('房本满五年', na=False)
            df.loc[unknown_mask & five_year_mask, '房屋年限'] = '满五年'
            
            # 检查是否包含"房本满两年"
            two_year_mask = df['房屋优势'].str.contains('房本满两年', na=False)
            df.loc[unknown_mask & two_year_mask, '房屋年限'] = '满两年'
            
            # 剩余的unknown改为"不满两年"
            still_unknown_mask = (df['房屋年限'] == 'unknown') & unknown_mask
            df.loc[still_unknown_mask, '房屋年限'] = '不满两年'
    
    return df
    
def extract_surrounding_features(df):
    """提取周边信息并映射为布尔值特征"""
    
    if '周边配套' in df.columns:
        # 创建与房价相关的新特征列，初始值为False
        features = {
            '医院配套': ['医院', '医疗', '诊所', '卫生院'],
            '公园绿地': ['公园', '绿地', '绿化', '景观'],
            '商超便利': ['超市', '商场', '购物', '百货', '市场', '卖场'],
            '银行金融': ['银行', '工行', '建行', '农行', '中行', '金融'],
            '餐饮美食': ['餐饮', '饭店', '餐厅', '酒楼', '美食', '快餐'],
            '学校教育': ['学校', '幼儿园', '小学', '中学', '大学', '教育'],
            '休闲娱乐': ['电影院', '健身', '会所', '娱乐', '广场', '体育']
        }
        
        # 初始化所有新列为False
        for feature_name in features.keys():
            df[feature_name] = False
        
        # 对每个特征进行关键词匹配
        for feature_name, keywords in features.items():
            # 构建正则表达式模式，匹配任意关键词
            pattern = '|'.join(keywords)
            mask = df['周边配套'].str.contains(pattern, na=False, case=False)
            df.loc[mask, feature_name] = True
    
    return df

def extract_facilities(df):
    """提取配套设施信息到布尔值列"""
    
    if '配套设施' in df.columns:
        # 定义所有可能的设施
        facilities = ['洗衣机', '空调', '衣柜', '电视', '冰箱', '热水器', '床', '暖气', '宽带', '天然气']
        
        # 为每个设施创建新列
        for facility in facilities:
            df[facility] = False
        for idx, value in df['配套设施'].items():
            if value != '无介绍信息' and pd.notna(value):
                for facility in facilities:
                    if facility in str(value):
                        df.at[idx, facility] = True
        
        # 删除原列
        df = df.drop(columns=['配套设施'])
    
    return df

def convert_binary_to_bool(df, columns=None):
    """将二值特征转换为布尔值"""
    for col in columns:
        if col in df.columns:
            unique_vals = df[col].dropna().unique()
            if len(unique_vals) == 2:
                # 手动指定映射，确保业务逻辑正确
                if '有' in unique_vals and '无' in unique_vals:
                    mapping = {'无': False, '有': True}
                elif '精装修' in unique_vals and '非精装修' in unique_vals:
                    mapping = {'非精装修': False, '精装修': True}
                elif '非共有' in unique_vals and '共有' in unique_vals:
                    mapping = {'非共有': False, '共有': True}
                else:
                    # 默认映射：按字母顺序或出现频率
                    mapping = {unique_vals[0]: False, unique_vals[1]: True}
                
                df[col] = df[col].map(mapping).astype(bool)
    
    return df

def create_binning_features(df):
    """创建分箱特征"""
    # 建筑年代分箱
    if '建筑年代' in df.columns:
        df['建筑年代分箱'] = pd.cut(df['建筑年代'], 
                                    bins=[0, 1980, 1990, 2000, 2010, 2020, 2030],
                                    labels=['老旧', '80年代', '90年代', '00年代', '10年代', '新建筑'])

    return df

def encode_categorical_features(df):
    """编码分类变量 - 指定特定列"""
    # 明确指定需要编码的列
    columns_to_encode = [
        '建筑年代分箱',
        '城市',
        '所在楼层', 
        '建筑结构',
        '装修情况',
        '房屋用途',
        '房屋年限',
        '供水',
        '供电',
        '用水',
        '用电',
        '付款方式',
        '租赁方式'
    ]
    
    for col in columns_to_encode:
        if col in df.columns:
            dummies = pd.get_dummies(df[col], prefix=col, prefix_sep='_')
            df = pd.concat([df, dummies], axis=1)
    
    return df

##### a. 处理构建Price预测模型的可能特征变量

In [23]:
X_train_price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83096 entries, 20486 to 77652
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      83096 non-null  int64  
 1   房屋户型    83096 non-null  object 
 2   所在楼层    83096 non-null  object 
 3   建筑面积    83096 non-null  float64
 4   房屋朝向    83096 non-null  object 
 5   建筑结构    83096 non-null  object 
 6   装修情况    83096 non-null  object 
 7   配备电梯    83096 non-null  object 
 8   房屋用途    83096 non-null  object 
 9   房屋年限    83096 non-null  object 
 10  产权所属    83096 non-null  object 
 11  房屋优势    83096 non-null  object 
 12  周边配套    83096 non-null  object 
 13  交通出行    83096 non-null  object 
 14  lon     83096 non-null  float64
 15  lat     83096 non-null  float64
 16  年份      83096 non-null  float64
 17  建筑年代    83096 non-null  float64
 18  房屋总数    83096 non-null  float64
 19  楼栋总数    83096 non-null  float64
 20  绿 化 率   83096 non-null  float64
 21  容 积 率   83096 non-null  float64
 22 

In [24]:
import matplotlib.pyplot as plt

m = X_train_price['建筑面积']

# 原始分布
plt.figure(figsize=(6,2))
plt.subplot(1,2,1)
m.hist(bins=30)
plt.title('原始数据')

# 对数化后分布
plt.subplot(1,2,2)
import numpy as np
np.log1p(m[m>0]).hist(bins=30)   # log1p 防 0 值
plt.title('log1p 后')
plt.show()

<Figure size 432x144 with 2 Axes>

In [25]:
def price_feature_engineering(df):
    """
    完整的特征工程处理流程
    """
    df_processed = df.copy()
    
    # 1. 楼层信息处理
    df_processed = process_floor_info(df_processed)

    # 2. 户型信息处理
    df_processed = extract_room_layout(df_processed)

    # 3. 交通信息处理
    df_processed = extract_transport_features(df_processed)

    # 4. 周边设施信息处理
    df_processed = extract_surrounding_features(df_processed)

    # 5. 朝向信息处理
    df_processed = extract_orientation_features(df_processed)

    # 6. 房屋年限信息补充
    df_processed = complete_property_age(df_processed)

    # 7. 房屋用途分类
    df_processed = classify_property_usage(df_processed)
    
    # 8. 数值分箱
    df_processed = create_binning_features(df_processed)

    # 9.处理二值变量
    df_processed.rename(columns={'产权所属': '产权共有'}, inplace=True)
    df_processeed = convert_binary_to_bool(df_processed, columns=['配备电梯','产权共有'])

    # 10. 编码处理
    df_processed = encode_categorical_features(df_processed)
    
    # 11. 对数项
    df_processed['log_建筑面积'] = np.log1p(df_processed['建筑面积'])
    
    # 12. 多项式特征
    df_processed['建筑面积_squared'] = df_processed['建筑面积'] ** 2
    df_processed['建筑面积_cubed'] = df_processed['建筑面积'] ** 3
    df_processed['容积率_squared'] = df_processed['容 积 率'] ** 2

    # 13. 交互项
    df_processed['面积_物业费_交互'] = df_processed['建筑面积'] * df_processed['物 业 费']
    df_processed['面积_容积率_交互'] = df_processed['建筑面积'] * df_processed['容 积 率']
    df_processed['绿化率_容积率_交互'] = df_processed['绿 化 率'] * df_processed['容 积 率']
    
    return df_processed


In [26]:
# 应用特征工程
X_train_price_pro = price_feature_engineering(X_train_price).select_dtypes(include=['float64','bool'])
X_test_price_pro = price_feature_engineering(X_test_price).select_dtypes(include=['float64','bool'])
print(X_train_price_pro.shape)

(83096, 95)


In [27]:
X_train_price_pro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83096 entries, 20486 to 77652
Data columns (total 95 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   建筑面积          83096 non-null  float64
 1   配备电梯          83096 non-null  bool   
 2   产权共有          83096 non-null  bool   
 3   lon           83096 non-null  float64
 4   lat           83096 non-null  float64
 5   年份            83096 non-null  float64
 6   建筑年代          83096 non-null  float64
 7   房屋总数          83096 non-null  float64
 8   楼栋总数          83096 non-null  float64
 9   绿 化 率         83096 non-null  float64
 10  容 积 率         83096 non-null  float64
 11  物 业 费         83096 non-null  float64
 12  燃气费           83096 non-null  float64
 13  停车位           83096 non-null  float64
 14  停车费用          83096 non-null  float64
 15  室             83096 non-null  float64
 16  厅             83096 non-null  float64
 17  卫             83096 non-null  float64
 18  厨             83096 non-nul

##### b. 处理构建Rent预测模型的可能特征变量

In [28]:
X_train_rent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79119 entries, 66850 to 77652
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      79119 non-null  int64  
 1   户型      79119 non-null  object 
 2   面积      79119 non-null  float64
 3   朝向      79119 non-null  object 
 4   付款方式    79119 non-null  object 
 5   租赁方式    79119 non-null  object 
 6   电梯      79119 non-null  object 
 7   用水      79119 non-null  object 
 8   用电      79119 non-null  object 
 9   燃气      79119 non-null  object 
 10  配套设施    79119 non-null  object 
 11  lon     79119 non-null  float64
 12  lat     79119 non-null  float64
 13  年份      79119 non-null  float64
 14  建筑年代    79119 non-null  float64
 15  房屋总数    79119 non-null  float64
 16  楼栋总数    79119 non-null  float64
 17  绿 化 率   79119 non-null  float64
 18  容 积 率   79119 non-null  float64
 19  物 业 费   79119 non-null  float64
 20  建筑结构    79119 non-null  object 
 21  燃气费     79119 non-null  float64
 22 

In [29]:
def rent_feature_engineering(df):
    """
    完整的特征工程处理流程
    """
    df_processed = df.copy()
    
    # 1. 楼层信息处理
    df_processed = process_floor_info(df_processed)

    # 2. 户型信息处理
    df_processed = extract_room_layout(df_processed)

    # 3. 房屋用途分类
    df_processed = classify_property_usage(df_processed)
    
    # 4. 配套设施信息处理
    df_processed = extract_facilities(df_processed)

    # 5. 朝向信息处理
    df_processed = extract_orientation_features(df_processed)

    # 6. 建筑结构处理
    structure_types = ['板楼', '平房', '塔板结合', '塔楼']
    df_processed.loc[~df_processed['建筑结构'].isin(structure_types), '建筑结构'] = '综合'

    # 7.处理二值变量
    df_processed = convert_binary_to_bool(df_processed, columns=['电梯','燃气','装修'])

    # 8. 数值分箱
    df_processed = create_binning_features(df_processed)
    
    # 9. 编码处理
    df_processed = encode_categorical_features(df_processed)
    
    # 10. 对数项
    df_processed['log_面积'] = np.log1p(df_processed['面积'])
    
    # 11. 多项式特征
    df_processed['面积_squared'] = df_processed['面积'] ** 2
    df_processed['面积_cubed'] = df_processed['面积'] ** 3
    df_processed['容积率_squared'] = df_processed['容 积 率'] ** 2

    # 12. 交互项
    df_processed['面积_物业费_交互'] = df_processed['面积'] * df_processed['物 业 费']
    df_processed['面积_容积率_交互'] = df_processed['面积'] * df_processed['容 积 率']
    df_processed['绿化率_容积率_交互'] = df_processed['绿 化 率'] * df_processed['容 积 率']
    
    return df_processed


In [30]:
# 应用特征工程
X_train_rent_pro = rent_feature_engineering(X_train_rent).select_dtypes(include=['float64','bool'])
X_test_rent_pro = rent_feature_engineering(X_test_rent).select_dtypes(include=['float64','bool'])
print(X_train_rent_pro.shape)

(79119, 80)


In [31]:
X_train_rent_pro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79119 entries, 66850 to 77652
Data columns (total 80 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   面积           79119 non-null  float64
 1   电梯           79119 non-null  bool   
 2   燃气           79119 non-null  bool   
 3   lon          79119 non-null  float64
 4   lat          79119 non-null  float64
 5   年份           79119 non-null  float64
 6   建筑年代         79119 non-null  float64
 7   房屋总数         79119 non-null  float64
 8   楼栋总数         79119 non-null  float64
 9   绿 化 率        79119 non-null  float64
 10  容 积 率        79119 non-null  float64
 11  物 业 费        79119 non-null  float64
 12  燃气费          79119 non-null  float64
 13  停车位          79119 non-null  float64
 14  停车费用         79119 non-null  float64
 15  装修           79119 non-null  bool   
 16  室            79119 non-null  float64
 17  厅            79119 non-null  float64
 18  卫            79119 non-null  float64
 19  厨    

#### 2.2 特征变量选择

In [32]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_selection(X_train, y_train, X_test):
    """改进的特征选择函数，避免双重特征选择"""
    
    # 1. 基于相关性的初步筛选
    corr_matrix = X_train.corrwith(y_train).abs()
    high_corr_features = corr_matrix[corr_matrix > 0.1].index.tolist()
    
    if not high_corr_features:
        high_corr_features = corr_matrix.nlargest(5).index.tolist()
    
    # 2. 检查共线性 (VIF分析)
    def check_vif(X):
        from statsmodels.stats.outliers_influence import variance_inflation_factor
        from statsmodels.tools.tools import add_constant
        
        # 直接转换布尔值为整数（仅用于VIF计算）
        X_for_vif = X.copy()
        bool_cols = X_for_vif.select_dtypes(include=['bool']).columns
        X_for_vif[bool_cols] = X_for_vif[bool_cols].astype(int)
        
        X_const = add_constant(X_for_vif)
        vif_data = pd.DataFrame()
        vif_data["feature"] = X_const.columns
        vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) 
                           for i in range(X_const.shape[1])]
        vif_data = vif_data[vif_data['feature'] != 'const']
        return vif_data.sort_values("VIF", ascending=False)
    
    # 对相关性筛选后的特征进行VIF分析
    vif_results = check_vif(X_train[high_corr_features])
    
    # 3. 处理高VIF特征 (VIF >= 10)
    high_vif_features = vif_results[vif_results['VIF'] >= 10]['feature'].tolist()
    
    if high_vif_features:
        features_to_keep = high_corr_features.copy()
        
        for high_vif_feature in high_vif_features:
            if high_vif_feature not in features_to_keep:
                continue
                
            corr_with_high_vif = X_train[features_to_keep].corrwith(
                X_train[high_vif_feature]).abs()
            
            collinear_group = corr_with_high_vif[corr_with_high_vif > 0.8].index.tolist()
            
            if len(collinear_group) > 1:
                group_corr_with_target = corr_matrix[collinear_group]
                best_feature = group_corr_with_target.idxmax()
                
                features_to_remove = [f for f in collinear_group if f != best_feature]
                for feature in features_to_remove:
                    if feature in features_to_keep:
                        features_to_keep.remove(feature)
    else:
        features_to_keep = high_corr_features.copy()

    print(f"最终选择特征数量: {len(features_to_keep)}")
    print(f"最终特征: {features_to_keep}")

    return X_train[features_to_keep], X_test[features_to_keep]


# 应用price特征选择
Xp_train_selected, Xp_test_selected = feature_selection(X_train_price_pro, y_train_price, X_test_price_pro)
# 应用rent特征选择
Xr_train_selected, Xr_test_selected = feature_selection(X_train_rent_pro, y_train_rent, X_test_rent_pro)

最终选择特征数量: 29
最终特征: ['lon', 'lat', '建筑年代', '燃气费', '停车费用', '室', '厅', '卫', '地铁', '北', '南北通透', '建筑年代分箱_00年代', '建筑年代分箱_10年代', '城市_0', '城市_1', '城市_2', '城市_4', '城市_6', '所在楼层_底层', '所在楼层_顶层', '装修情况_精装', '房屋用途_商住混合', '房屋用途_豪宅', '房屋年限_不满两年', '房屋年限_满五年', 'log_建筑面积', '建筑面积_squared', '面积_物业费_交互', '面积_容积率_交互']
最终选择特征数量: 28
最终特征: ['lon', 'lat', '建筑年代', '房屋总数', '物 业 费', '燃气费', '停车位', '停车费用', '室', '厅', '空调', '电视', '暖气', '建筑年代分箱_10年代', '城市_0', '城市_1', '城市_2', '城市_3', '城市_4', '城市_7', '城市_8', '建筑结构_塔楼', '建筑结构_板楼', '租赁方式_合租', 'log_面积', '面积_squared', '面积_物业费_交互', '面积_容积率_交互']


In [33]:
Xp_train_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83096 entries, 20486 to 77652
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lon           83096 non-null  float64
 1   lat           83096 non-null  float64
 2   建筑年代          83096 non-null  float64
 3   燃气费           83096 non-null  float64
 4   停车费用          83096 non-null  float64
 5   室             83096 non-null  float64
 6   厅             83096 non-null  float64
 7   卫             83096 non-null  float64
 8   地铁            83096 non-null  bool   
 9   北             83096 non-null  bool   
 10  南北通透          83096 non-null  bool   
 11  建筑年代分箱_00年代   83096 non-null  bool   
 12  建筑年代分箱_10年代   83096 non-null  bool   
 13  城市_0          83096 non-null  bool   
 14  城市_1          83096 non-null  bool   
 15  城市_2          83096 non-null  bool   
 16  城市_4          83096 non-null  bool   
 17  城市_6          83096 non-null  bool   
 18  所在楼层_底层       83096 non-nul

In [34]:
Xr_train_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79119 entries, 66850 to 77652
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lon          79119 non-null  float64
 1   lat          79119 non-null  float64
 2   建筑年代         79119 non-null  float64
 3   房屋总数         79119 non-null  float64
 4   物 业 费        79119 non-null  float64
 5   燃气费          79119 non-null  float64
 6   停车位          79119 non-null  float64
 7   停车费用         79119 non-null  float64
 8   室            79119 non-null  float64
 9   厅            79119 non-null  float64
 10  空调           79119 non-null  bool   
 11  电视           79119 non-null  bool   
 12  暖气           79119 non-null  bool   
 13  建筑年代分箱_10年代  79119 non-null  bool   
 14  城市_0         79119 non-null  bool   
 15  城市_1         79119 non-null  bool   
 16  城市_2         79119 non-null  bool   
 17  城市_3         79119 non-null  bool   
 18  城市_4         79119 non-null  bool   
 19  城市_7 

#### 2.3 处理异常值

In [35]:
print(Xp_train_selected.shape)
print(Xr_train_selected.shape)

(83096, 29)
(79119, 28)


In [36]:
def remove_outliers_zscore_strict(X_train, y_train, X_test, y_test, threshold=3, min_outliers=2):
    """要求至少min_outliers个特征为异常值才删除样本"""
    
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns
    z_scores = np.abs((X_train[numeric_cols] - X_train[numeric_cols].mean()) / X_train[numeric_cols].std())
    
    # 计算每个样本的异常特征数量
    outlier_counts = (z_scores > threshold).sum(axis=1)
    
    # 只删除异常特征数量达到阈值的样本
    outliers_mask = outlier_counts >= min_outliers
    
    X_train_clean = X_train[~outliers_mask]
    y_train_clean = y_train[~outliers_mask]
    
    print(f"移除异常值: {outliers_mask.sum()} 个样本")
    print(f"清洗后训练集形状: {X_train_clean.shape}")
    
    return X_train_clean, y_train_clean, X_test, y_test

# 要求至少2个特征为异常值才删除样本
Xp_train_clean, yp_train_clean, Xp_test_clean, yp_test_clean = remove_outliers_zscore_strict(
    Xp_train_selected, y_train_price, Xp_test_selected, y_test_price, threshold=3, min_outliers=2
)
Xr_train_clean, yr_train_clean, Xr_test_clean, yr_test_clean = remove_outliers_zscore_strict(
    Xr_train_selected, y_train_rent, Xr_test_selected, y_test_rent, threshold=3, min_outliers=2
)

移除异常值: 1484 个样本
清洗后训练集形状: (81612, 29)
移除异常值: 1480 个样本
清洗后训练集形状: (77639, 28)


In [37]:
# def remove_outliers_zscore_strict(X_train, y_train, X_test, y_test, threshold=3):
#     """只要有1个特征为异常值就删除样本"""
    
#     numeric_cols = X_train.select_dtypes(include=[np.number]).columns
#     z_scores = np.abs((X_train[numeric_cols] - X_train[numeric_cols].mean()) / X_train[numeric_cols].std())
    
#     # 计算每个样本是否有异常特征
#     outliers_mask = (z_scores > threshold).any(axis=1)
    
#     X_train_clean = X_train[~outliers_mask]
#     y_train_clean = y_train[~outliers_mask]
    
#     return X_train_clean, y_train_clean, X_test, y_test

# # 只要有1个特征为异常值就删除样本
# Xp_train_clean, yp_train_clean, Xp_test_clean, yp_test_clean = remove_outliers_zscore_strict(
#     Xp_train_selected, y_train_price, Xp_test_selected, y_test_price, threshold=3
# )
# Xr_train_clean, yr_train_clean, Xr_test_clean, yr_test_clean = remove_outliers_zscore_strict(
#     Xr_train_selected, y_train_rent, Xr_test_selected, y_test_rent, threshold=3
# )

### 4. Modeling (Individual):  

  - You must use linear models: **Linear Regression, Lasso, Ridge regression, Elastic Net**  
  - Fine-tune model: try to improve your model by 1) add and drop features, 2) add non-linearity and interactions, 3) change hyperparameters (L1 and L2 regularizers) of model  
  - You can use sklearn.linear_model and sklearn.model_selection.GridSearchCV to fine-tune your model  
  - Report both your: in-sample, out of sample and [6 folds cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) model performance for your OLS, LASSO, Ridge and best model (please use Mean Absolute Error, RMAE to report the performance). Also, please report the total number of predictions after you remove the outlier of the sample.  

#### 4.1 处理需要预测price/rent的输入特征

In [38]:
# 处理df_te_price
df_te_price_useful = df_te_price.drop(columns=price_cols_to_drop) 
df_te_price_preprocessed = preprocess_data(df_te_price_useful)
X_price_te = df_te_price_preprocessed.drop(columns = unwanted_price_fe)
X_price_te_pro = price_feature_engineering(X_price_te).select_dtypes(include=['float64','bool'])
Xp_to_pred = X_price_te_pro[Xp_train_selected.columns]
print('\n预测price的输入特征处理完成！\n')

# 处理df_te_rent
df_te_rent_useful = df_te_rent.drop(columns=rent_cols_to_drop) 
df_te_rent_useful['装修'] = df_te_rent['装修'].fillna('非精装修') # 补一下装修信息 
payway = ["半年付价", "季付价", "年付价", "双月付价", "月付价"]
df_te_rent_useful.loc[~df_te_rent_useful['付款方式'].isin(payway), '付款方式'] = '季付价' # 删点怪东西
df_te_rent_preprocessed = preprocess_data(df_te_rent_useful)
X_rent_te = df_te_rent_preprocessed.drop(columns = unwanted_rent_fe)
X_rent_te_pro = rent_feature_engineering(X_rent_te).select_dtypes(include=['float64','bool'])
Xr_to_pred = X_rent_te_pro[Xr_train_selected.columns]
print('\n预测rent的输入特征处理完成！\n')


开始数据预处理...
提取数字特征...
正在进行地理聚类...
开始填充缺失值...
房屋户型列已完成填充，共填充14个缺失值
建筑结构列已完成填充，共填充14个缺失值
装修情况列已完成填充，共填充14个缺失值
梯户比例列已完成填充，共填充635个缺失值
配备电梯列已完成填充，共填充4092个缺失值
上次交易列已完成填充，共填充7241个缺失值
房屋年限列已完成填充，共填充11296个缺失值
房屋优势列已完成填充，共填充6379个缺失值
周边配套列已完成填充，共填充15031个缺失值
交通出行列已完成填充，共填充15358个缺失值
物业类别列已完成填充，共填充7533个缺失值
建筑年代列已完成填充，共填充9406个缺失值
开发商列已完成填充，共填充10592个缺失值
房屋总数列已完成填充，共填充3715个缺失值
楼栋总数列已完成填充，共填充3715个缺失值
物业公司列已完成填充，共填充10942个缺失值
绿 化 率列已完成填充，共填充9180个缺失值
容 积 率列已完成填充，共填充9303个缺失值
物 业 费列已完成填充，共填充8489个缺失值
产权描述列已完成填充，共填充7533个缺失值
供水列已完成填充，共填充8195个缺失值
供电列已完成填充，共填充8177个缺失值
燃气费列已完成填充，共填充9157个缺失值
停车位列已完成填充，共填充9571个缺失值
停车费用列已完成填充，共填充11387个缺失值

预处理完成!
总样本数: 34017
地理区域数: 100

预测price的输入特征处理完成！

开始数据预处理...
提取数字特征...
正在进行地理聚类...
开始填充缺失值...
用水列已完成填充，共填充2273个缺失值
用电列已完成填充，共填充2227个缺失值
燃气列已完成填充，共填充591个缺失值
租期列已完成填充，共填充5175个缺失值
配套设施列已完成填充，共填充3694个缺失值
区县列已完成填充，共填充925个缺失值
板块列已完成填充，共填充936个缺失值
物业类别列已完成填充，共填充2625个缺失值
建筑年代列已完成填充，共填充3381个缺失值
开发商列已完成填充，共填充3303个缺失值
房屋总数列已完成填充，共填充923个缺失值
楼栋总数列已完成填充，共填充923个缺失值
物业公司列已完成填充，共填充3479个缺失值
绿 化 率列已完成填

In [39]:
def are_dataframes_identical(df1, df2):
    """检查两个DataFrame的列名和数据类型是否完全相同"""
    result = df1.columns.equals(df2.columns) and df1.dtypes.equals(df2.dtypes)
    print(f"两个数据表列名和数据类型完全相同: {result}")

are_dataframes_identical(Xp_train_selected, Xp_to_pred)
are_dataframes_identical(Xr_train_selected, Xr_to_pred)

两个数据表列名和数据类型完全相同: True
两个数据表列名和数据类型完全相同: True


##### 4.2 导入必要的库

In [40]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
import time
import joblib
import os

# 设置并行计算和显示选项
pd.set_option('display.max_columns', None)

# 创建存储目录
output_dir = "/home/mw/project/select"
os.makedirs(output_dir, exist_ok=True)
print(f"输出目录: {output_dir}")

输出目录: /home/mw/project/select


#### 4.3 定义评估函数和模型配置

In [41]:
def evaluate_model(model, X_train, y_train_orig, X_test, y_test_orig, 
                  model_name, use_log=False, cv_folds=6):
    """统一的模型评估函数，支持对数变换"""
    
    print(f"正在评估 {model_name}...")
    
    # 处理对数变换
    if use_log:
        y_train_eval = np.log1p(y_train_orig)
        y_test_eval = np.log1p(y_test_orig)
    else:
        y_train_eval = y_train_orig.copy()
        y_test_eval = y_test_orig.copy()
    
    # In-sample 评估
    start_time = time.time()
    y_train_pred = model.predict(X_train)
    if use_log:
        y_train_pred_orig = np.expm1(y_train_pred)
        train_mae = mean_absolute_error(y_train_orig, y_train_pred_orig)
    else:
        train_mae = mean_absolute_error(y_train_orig, y_train_pred)
    train_rmae = train_mae / np.mean(y_train_orig)
    train_time = time.time() - start_time
    
    # Out-of-sample 评估
    start_time = time.time()
    y_test_pred = model.predict(X_test)
    if use_log:
        y_test_pred_orig = np.expm1(y_test_pred)
        test_mae = mean_absolute_error(y_test_orig, y_test_pred_orig)
    else:
        test_mae = mean_absolute_error(y_test_orig, y_test_pred)
    test_rmae = test_mae / np.mean(y_test_orig)
    test_time = time.time() - start_time
    
    # 交叉验证评估
    start_time = time.time()
    if use_log:
        # 对于对数变换，我们需要自定义评分函数
        from sklearn.metrics import make_scorer
        def mae_log_scorer(y_true_log, y_pred_log):
            y_true_orig = np.expm1(y_true_log)
            y_pred_orig = np.expm1(y_pred_log)
            return mean_absolute_error(y_true_orig, y_pred_orig)  # 为什么不用加符号啊尼玛，但能运行就算了
        
        mae_scorer = make_scorer(mae_log_scorer, greater_is_better=False)
        cv_scores_mae = -cross_val_score(model, X_train, y_train_eval, 
                                        cv=cv_folds, scoring=mae_scorer, n_jobs=-1)
    else:
        cv_scores_mae = -cross_val_score(model, X_train, y_train_eval, 
                                        cv=cv_folds, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    cv_mae = cv_scores_mae.mean()
    cv_rmae = cv_mae / np.mean(y_train_orig)
    cv_time = time.time() - start_time
    
    print(f"{model_name} 评估完成 - 训练: {train_time:.1f}s, 测试: {test_time:.1f}s, CV: {cv_time:.1f}s")
    
    return {
        'model_name': model_name,
        'in_sample_mae': train_mae,
        'in_sample_rmae': train_rmae,
        'out_sample_mae': test_mae,
        'out_sample_rmae': test_rmae,
        'cv_mae': cv_mae,
        'cv_rmae': cv_rmae,
        'model': model
    }

def create_prediction_with_log(model, X_data, use_log=False):
    """统一的预测函数，支持对数变换"""
    predictions = model.predict(X_data)
    if use_log:
        return np.expm1(predictions)
    else:
        return predictions

def create_kaggle_submission(model, X_to_pred, ids, model_name, use_log=False):
    """创建Kaggle提交文件"""
    predictions = create_prediction_with_log(model, X_to_pred, use_log)
    submission_df = pd.DataFrame({
        'ID': ids,
        'Price_pred': predictions
    })
    return submission_df

#### 4.4 定义模型训练函数

In [42]:
def train_linear_models(X_train, y_train, X_test, y_test, X_to_pred, ids, data_type="price", use_log_transform=False):
    """训练所有线性模型并返回结果"""
    
    print(f"开始训练 {data_type} 模型...")
    if use_log_transform:
        print("使用对数变换")
    
    results = []
    submission_dfs = {}
    
    # 数据标准化（对正则化模型很重要）
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_to_pred_scaled = scaler.transform(X_to_pred)
    
    # 处理对数变换
    if use_log_transform:
        y_train_transformed = np.log1p(y_train)
    else:
        y_train_transformed = y_train.copy()
    
    # 模型参数网格
    param_grids = {
        'Lasso': {
            'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
            'max_iter': [1000, 5000],
            'selection': ['cyclic']
        },
        'Ridge': {
            'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
            'max_iter': [1000, 5000]
        },
        'ElasticNet': {
            'alpha': [0.001, 0.01, 0.1, 1],
            'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'max_iter': [1000, 5000]
        }
    }
    
    # 1. Linear Regression (OLS)
    print("训练 Linear Regression...")
    lr_model = LinearRegression(n_jobs=-1)
    lr_model.fit(X_train, y_train_transformed)
    lr_results = evaluate_model(lr_model, X_train, y_train, X_test, y_test, 
                               "OLS", use_log_transform)
    results.append(lr_results)
    submission_dfs['OLS'] = create_kaggle_submission(lr_model, X_to_pred, ids, 
                                                   "OLS", use_log_transform)
    
    # 2. Lasso Regression
    print("训练 Lasso Regression...")
    lasso = Lasso(random_state=111)
    lasso_grid = GridSearchCV(lasso, param_grids['Lasso'], cv=6, scoring='neg_mean_absolute_error', 
                             n_jobs=-1, verbose=0)
    lasso_grid.fit(X_train_scaled, y_train_transformed)
    print(f"Lasso 最佳参数: {lasso_grid.best_params_}")
    lasso_results = evaluate_model(lasso_grid.best_estimator_, X_train_scaled, y_train, 
                                  X_test_scaled, y_test, "LASSO", use_log_transform)
    results.append(lasso_results)
    submission_dfs['LASSO'] = create_kaggle_submission(lasso_grid.best_estimator_, X_to_pred_scaled, ids, 
                                                     "LASSO", use_log_transform)
    
    # 3. Ridge Regression
    print("训练 Ridge Regression...")
    ridge = Ridge(random_state=111)
    ridge_grid = GridSearchCV(ridge, param_grids['Ridge'], cv=6, scoring='neg_mean_absolute_error', 
                             n_jobs=-1, verbose=0)
    ridge_grid.fit(X_train_scaled, y_train_transformed)
    print(f"Ridge 最佳参数: {ridge_grid.best_params_}")
    ridge_results = evaluate_model(ridge_grid.best_estimator_, X_train_scaled, y_train, 
                                  X_test_scaled, y_test, "Ridge", use_log_transform)
    results.append(ridge_results)
    submission_dfs['Ridge'] = create_kaggle_submission(ridge_grid.best_estimator_, X_to_pred_scaled, ids, 
                                                     "Ridge", use_log_transform)
    
    # 4. Elastic Net
    print("训练 Elastic Net...")
    elastic = ElasticNet(random_state=111)
    elastic_grid = GridSearchCV(elastic, param_grids['ElasticNet'], cv=6, scoring='neg_mean_absolute_error', 
                               n_jobs=-1, verbose=0)
    elastic_grid.fit(X_train_scaled, y_train_transformed)
    print(f"ElasticNet 最佳参数: {elastic_grid.best_params_}")
    elastic_results = evaluate_model(elastic_grid.best_estimator_, X_train_scaled, y_train, 
                                    X_test_scaled, y_test, "ElasticNet", use_log_transform)
    results.append(elastic_results)
    submission_dfs['ElasticNet'] = create_kaggle_submission(elastic_grid.best_estimator_, X_to_pred_scaled, ids, 
                                                          "ElasticNet", use_log_transform)
    
    
    # 确定最佳模型
    best_model_idx = np.argmin([result['out_sample_mae'] for result in results])
    best_model_name = results[best_model_idx]['model_name']
    best_model = results[best_model_idx]['model']
    
    print(f"最佳模型: {best_model_name}")
    
    return results, submission_dfs, best_model_name, best_model

#### 4.5 执行价格数据建模

In [43]:
import matplotlib.pyplot as plt

a = yp_train_clean

# 原始分布
plt.figure(figsize=(6,2))
plt.subplot(1,2,1)
a.hist(bins=30)
plt.title('原始数据')

# 对数化后分布
plt.subplot(1,2,2)
import numpy as np
np.log1p(a[a>0]).hist(bins=30)   # log1p 防 0 值
plt.title('log1p 后')
plt.show()

<Figure size 432x144 with 2 Axes>

In [44]:
print("=" * 50)
print("开始价格数据建模（使用对数变换）")
print("=" * 50)

price_ids = df_te_price['ID']

price_results, price_submissions, price_best_model_name, price_best_model = train_linear_models(
    Xp_train_clean, yp_train_clean, 
    Xp_test_clean, yp_test_clean,
    Xp_to_pred, price_ids, "price", use_log_transform=True
)

开始价格数据建模（使用对数变换）
开始训练 price 模型...
使用对数变换
训练 Linear Regression...
正在评估 OLS...
OLS 评估完成 - 训练: 0.2s, 测试: 0.0s, CV: 1.8s
训练 Lasso Regression...
Lasso 最佳参数: {'alpha': 0.001, 'max_iter': 1000, 'selection': 'cyclic'}
正在评估 LASSO...
LASSO 评估完成 - 训练: 0.4s, 测试: 0.2s, CV: 0.9s
训练 Ridge Regression...
Ridge 最佳参数: {'alpha': 0.001, 'max_iter': 1000}
正在评估 Ridge...
Ridge 评估完成 - 训练: 0.2s, 测试: 0.1s, CV: 0.4s
训练 Elastic Net...
ElasticNet 最佳参数: {'alpha': 0.001, 'l1_ratio': 0.1, 'max_iter': 1000}
正在评估 ElasticNet...
ElasticNet 评估完成 - 训练: 0.2s, 测试: 0.1s, CV: 1.2s
最佳模型: OLS


#### 4.6 执行租金数据建模

In [45]:
import matplotlib.pyplot as plt

s = yr_train_clean

# 原始分布
plt.figure(figsize=(6,2))
plt.subplot(1,2,1)
s.hist(bins=30)
plt.title('原始数据')

# 对数化后分布
plt.subplot(1,2,2)
import numpy as np
np.log1p(s[s>0]).hist(bins=30)   # log1p 防 0 值
plt.title('log1p 后')
plt.show()

<Figure size 432x144 with 2 Axes>

In [46]:
print("=" * 50)
print("开始租金数据建模（使用对数变换）")
print("=" * 50)

rent_ids = df_te_rent['ID']

rent_results, rent_submissions, rent_best_model_name, rent_best_model = train_linear_models(
    Xr_train_clean, yr_train_clean, 
    Xr_test_clean, yr_test_clean,
    Xr_to_pred, rent_ids, "rent", use_log_transform=True  # 租金也使用对数变换
)

开始租金数据建模（使用对数变换）
开始训练 rent 模型...
使用对数变换
训练 Linear Regression...
正在评估 OLS...
OLS 评估完成 - 训练: 0.1s, 测试: 0.0s, CV: 0.5s
训练 Lasso Regression...


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.882e+00, tolerance: 3.483e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.494e+00, tolerance: 3.472e+00
  model = cd_fast.enet_coordinate_descent(


Lasso 最佳参数: {'alpha': 0.001, 'max_iter': 5000, 'selection': 'cyclic'}
正在评估 LASSO...
LASSO 评估完成 - 训练: 0.3s, 测试: 0.0s, CV: 5.1s
训练 Ridge Regression...
Ridge 最佳参数: {'alpha': 0.001, 'max_iter': 1000}
正在评估 Ridge...
Ridge 评估完成 - 训练: 0.0s, 测试: 0.0s, CV: 0.5s
训练 Elastic Net...


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.141e+02, tolerance: 3.483e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.967e+02, tolerance: 3.472e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.413e+02, tolerance: 3.482e+00

ElasticNet 最佳参数: {'alpha': 0.001, 'l1_ratio': 0.1, 'max_iter': 5000}
正在评估 ElasticNet...
ElasticNet 评估完成 - 训练: 0.0s, 测试: 0.1s, CV: 13.4s
最佳模型: OLS


#### 4.7 生成结果表格

In [47]:
def create_results_table(results, data_type):
    """创建结果汇总表格"""
    df = pd.DataFrame(results)
    result_table = df[['model_name', 'in_sample_mae', 'in_sample_rmae', 
                      'out_sample_mae', 'out_sample_rmae', 'cv_mae', 'cv_rmae']].copy()
    
    result_table.columns = ['Model', 'In sample MAE', 'In sample RMAE', 
                           'Out of sample MAE', 'Out of sample RMAE', 
                           'CV MAE', 'CV RMAE']
    
    result_table['Kaggle Score'] = '待提交'
    
    print(f"\n{data_type} 模型性能汇总:")
    print("=" * 80)
    print(result_table.round(4))
    print("=" * 80)
    
    return result_table

# 生成价格结果表格
price_table = create_results_table(price_results, "价格")
# 生成租金结果表格  
rent_table = create_results_table(rent_results, "租金")


价格 模型性能汇总:
        Model  In sample MAE  In sample RMAE  Out of sample MAE  \
0         OLS    680571.8271          0.3165        732842.2789   
1       LASSO    681097.0925          0.3168        736198.7699   
2       Ridge    680571.8281          0.3165        732842.2844   
3  ElasticNet    680679.6097          0.3166        733562.6596   

   Out of sample RMAE       CV MAE  CV RMAE Kaggle Score  
0              0.3281  680942.4315   0.3167          待提交  
1              0.3296  681424.6162   0.3169          待提交  
2              0.3281  680942.4325   0.3167          待提交  
3              0.3284  681043.5499   0.3167          待提交  

租金 模型性能汇总:
        Model  In sample MAE  In sample RMAE  Out of sample MAE  \
0         OLS    138637.3933          0.2523        250782.7513   
1       LASSO    139785.0273          0.2544        256078.2210   
2       Ridge    138637.4109          0.2523        250782.9824   
3  ElasticNet    139541.7848          0.2539        256823.0427   

   Out of

#### 4.8 合并相同模型的price和rent预测并保存提交文件

In [48]:
def merge_price_rent_submissions(price_submissions, rent_submissions, model_name):
    """合并相同模型的price和rent预测"""
    price_df = price_submissions[model_name].copy()
    rent_df = rent_submissions[model_name].copy()
    
    price_df = price_df[['ID', 'Price_pred']]
    rent_df = rent_df[['ID', 'Price_pred']]
    
    merged_df = pd.concat([price_df, rent_df], axis=0, ignore_index=False)
    return merged_df

print("合并price和rent预测并保存提交文件...")

model_names = list(price_submissions.keys())

for model_name in model_names:
    if model_name in rent_submissions:
        merged_df = merge_price_rent_submissions(price_submissions, rent_submissions, model_name)
        filename = os.path.join(output_dir, f"submission_merged_{model_name}.csv")
        merged_df.to_csv(filename, index=False)
        print(f"已保存: {filename} (共 {len(merged_df)} 行)")

print(f"\n最佳价格模型: {price_best_model_name}")
print(f"最佳租金模型: {rent_best_model_name}")

合并price和rent预测并保存提交文件...
已保存: /home/mw/project/select/submission_merged_OLS.csv (共 43790 行)
已保存: /home/mw/project/select/submission_merged_LASSO.csv (共 43790 行)
已保存: /home/mw/project/select/submission_merged_Ridge.csv (共 43790 行)
已保存: /home/mw/project/select/submission_merged_ElasticNet.csv (共 43790 行)

最佳价格模型: OLS
最佳租金模型: OLS


#### 4.9 保存训练好的模型到指定路径

In [49]:
print("保存训练好的模型...")
joblib.dump(price_best_model, os.path.join(output_dir, 'best_price_model.pkl'))
joblib.dump(rent_best_model, os.path.join(output_dir, 'best_rent_model.pkl'))

print("所有模型训练和评估完成！")
print(f"所有文件保存在: {output_dir}")

保存训练好的模型...
所有模型训练和评估完成！
所有文件保存在: /home/mw/project/select
